<a href="https://colab.research.google.com/github/Dinhthixuanbinh/AIO/blob/main/c_expr_db_extract_frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Setup Kaggle
Tutorial: https://www.kaggle.com/code/vitouphy/how-to-upload-from-google-colab-to-kaggle-dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!sudo apt install tree 2>&1 > /dev/null
!pip install -q kaggle

In [ ]:
%%bash
mkdir -p ~/.kaggle
cp /content/drive/MyDrive/Workplaces/kaggle/kaggle.json ~/.kaggle
chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets init -p /content/drive/MyDrive/Workplaces/c-expr-frames

Data package template written to: /content/drive/MyDrive/Workplaces/c-expr-frames/dataset-metadata.json


## 2. Extract frames

In [ ]:
work_dir = '/content/drive/MyDrive/Workplaces/abaw6/c-expr-db'
!tree -R $work_dir

/content/drive/MyDrive/Workplaces/abaw6/c-expr-db
├── dataset-metadata.json
└── videos.zip

0 directories, 2 files


In [ ]:
data_dir = '/content/drive/MyDrive/Workplaces/c-expr-frames'

In [ ]:
import zipfile
import os

# Specify the path of the zip file and the target directory
zip_file_path = f'{work_dir}/videos.zip'
target_directory = data_dir

# Ensure the target directory exists, create if it doesn't
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

# Open the zip file and extract its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(target_directory)

print(f'Extracted all videos to {target_directory}')

Extracted all videos to /content/drive/MyDrive/Workplaces/c-expr-frames


In [ ]:
!tree -R $data_dir/frames

/content/drive/MyDrive/Workplaces/c-expr-frames/frames

0 directories, 0 files


In [ ]:
import cv2
import os


def extract_frames(video_path, target_folder, frame_count):
    # Determine the video number and prepare the target subfolder
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_number = video_name.split('.')[0]  # Assuming the video name is the number
    frame_target_folder = os.path.join(target_folder, video_number)

    # Create the target subfolder if it doesn't exist
    if not os.path.exists(frame_target_folder):
        os.makedirs(frame_target_folder)

    cap = cv2.VideoCapture(video_path)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Filename with natural numbers, zero-padded to 5 digits
        frame_filename = f"{frame_count:05d}.jpeg"
        frame_path = os.path.join(frame_target_folder, frame_filename)
        cv2.imwrite(frame_path, frame)

        frame_count += 1

    # Release the capture object
    cap.release()
    print(f"Extracted {frame_count - 1} frames from video {video_number} into folder {frame_target_folder}")
    return frame_count

source_directory = f'{data_dir}/videos'
target_directory = f'{data_dir}/frames'
frame_count = 1  # Start frame counting at 1

for video_file in os.listdir(source_directory):
    video_path = os.path.join(source_directory, video_file)
    if os.path.isfile(video_path) and not video_file.startswith('__MACOSX'):  # Skip macOS metadata folder
        frame_count = extract_frames(video_path, target_directory, frame_count)


Extracted 1465 frames from video 55 into folder /content/drive/MyDrive/Workplaces/c-expr-frames/frames/55
Extracted 1784 frames from video 41 into folder /content/drive/MyDrive/Workplaces/c-expr-frames/frames/41
Extracted 2552 frames from video 40 into folder /content/drive/MyDrive/Workplaces/c-expr-frames/frames/40
Extracted 3030 frames from video 54 into folder /content/drive/MyDrive/Workplaces/c-expr-frames/frames/54
Extracted 3593 frames from video 42 into folder /content/drive/MyDrive/Workplaces/c-expr-frames/frames/42
Extracted 3859 frames from video 56 into folder /content/drive/MyDrive/Workplaces/c-expr-frames/frames/56
Extracted 3934 frames from video 43 into folder /content/drive/MyDrive/Workplaces/c-expr-frames/frames/43
Extracted 4114 frames from video 47 into folder /content/drive/MyDrive/Workplaces/c-expr-frames/frames/47
Extracted 4556 frames from video 53 into folder /content/drive/MyDrive/Workplaces/c-expr-frames/frames/53
Extracted 4556 frames from video  into folder 

In [ ]:
# Push to Kaggle Using Kaggle API
!kaggle datasets init -p /content/drive/MyDrive/Workplaces/

Data package template written to: /content/drive/MyDrive/Workplaces/c-expr-frames/frames/dataset-metadata.json


In [ ]:
import json

metadata = {
  "title": "random-name-db-frames",
  "id": "hungstd/random-name-db-frames",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

json_object = json.dumps(metadata, indent=4)
with open(f"/content/drive/MyDrive/Workplaces/dataset-metadata.json", "w") as f:
  f.write(json_object)

In [ ]:
dataset_path = f'{data_dir}/frames'

In [ ]:
!kaggle datasets create -p $dataset_path --dir-mode zip